In [1]:
import os
from os.path import join as J
import pandas as pd
import numpy as np
import subprocess
from memory_fix import reduce_mem_usage

In [2]:
def REN(X):
    return X.rename(columns={"client_dk":'client_id'})

In [3]:
transactions_train = reduce_mem_usage( pd.read_csv('data/transactions_train.csv') )

Memory usage of dataframe is 668.09 MB
Memory usage after optimization is: 250.53 MB
Decreased by 62.5%


In [4]:
transactions_train = REN(transactions_train).rename(columns={"amount": "amount_rur"})

In [5]:
transactions_train.head(2)

,client_id,trans_date,amount_rur,small_group
0,43976,0,4.563000,2
1,8417,0,48.341999,0


In [6]:
def prep_date(data):
    data["trans_month"] = ((data["trans_date"] // 30) % 12 + 1)
    data["trans_year"] = ((data["trans_date"] // 365) + 1)

In [7]:
prep_date(transactions_train)
transactions_train.trans_year.value_counts()

1    20424426
2     1467553
Name: trans_year, dtype: int64

* client_id - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* small_group - категория покупки
* amount_rur - сумма транзакции

In [8]:
def get_first_last_transaction_date(data):
    ret = data.groupby('client_id')["trans_date"].agg(["min", "max"])
    ret.columns = ["first_transaction_date", "last_transaction_date"]
    return ret

In [9]:
def get_minmax_yearly_trans(data):
    ret = data.groupby(['client_id','trans_year'])["amount_rur"].agg(['count', 'sum'])\
        .groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

    ret.columns = ['client_id', 'yeartrans_max', 'yeartrans_min', 'yearcount_max', 'yearcount_min']
    ret["yearcount_diff"] = ret["yearcount_max"] - ret["yearcount_min"]
    return ret
get_minmax_yearly_trans(transactions_train)

,client_id,yeartrans_max,yeartrans_min,yearcount_max,yearcount_min,yearcount_diff
0,0,715,59,2.082540e+04,765.075012,20060.326172
1,2,1425,89,3.796030e+04,2368.613037,35591.683594
2,3,855,70,4.585046e+04,3164.098877,42686.359375
3,4,1319,93,4.647890e+04,5599.735840,40879.167969
4,7,713,77,9.010555e+04,7519.277832,82586.273438
...,...,...,...,...,...,...
24995,49992,1188,53,9.254634e+04,11967.704102,80578.640625
24996,49995,646,69,1.091684e+06,190214.468750,901469.625000
24997,49996,869,49,3.251385e+04,1235.448975,31278.402344
24998,49997,1266,50,3.293882e+04,1322.208984,31616.611328


In [10]:
def get_minmax_monthly_trans(data):
    ret = data.groupby(['client_id','trans_month'])["amount_rur"].agg(['count', 'sum'])\
        .groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

    ret.columns = ['client_id', 'monthtrans_max', 'monthtrans_min', 'monthcount_max', 'monthcount_min']
    ret["monthcount_diff"] = ret["monthcount_max"] - ret["monthcount_min"]
    return ret
get_minmax_monthly_trans(transactions_train)

,client_id,monthtrans_max,monthtrans_min,monthcount_max,monthcount_min,monthcount_diff
0,0,142,36,8429.054688,465.557007,7963.497559
1,2,208,85,7981.910156,1691.256958,6290.653320
2,3,111,57,5933.979980,1617.791016,4316.188965
3,4,204,35,10110.589844,655.034973,9455.554688
4,7,127,31,23676.210938,1625.359009,22050.851562
...,...,...,...,...,...,...
24995,49992,176,61,24913.185547,3598.437012,21314.748047
24996,49995,146,25,422985.312500,4729.438965,418255.875000
24997,49996,123,54,5124.808105,1848.009033,3276.799072
24998,49997,191,70,4887.059082,1405.685059,3481.374023


In [11]:
def get_row_with_max_col_val(df, col, group_col):
    idx = df.groupby(group_col)[col].transform(max) == df[col]
    return df[idx]

# Посмотрим на максимальные - минимальные затраты за 1 день

In [12]:
transactions_train.groupby(['client_id','trans_date'])["amount_rur"].agg(['count', 'sum']).agg(['max', 'min'])

,count,sum
max,96,459306.90625
min,1,0.00100


In [14]:
def get_stat(data, func='count'):
    if func == "count":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].count()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_'+str(i) for i in cat_counts.columns]
    
    elif func == "sum":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].sum()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_sum_'+str(i) for i in cat_counts.columns]
    
    elif func == 'std':
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].std()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_std_'+str(i) for i in cat_counts.columns]
        
    elif func == "median":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].median()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_median_'+str(i) for i in cat_counts.columns]
        
    return cat_counts



In [16]:
def preproc_data(transactions, is_train=True):
    
    prep_date(transactions)
    get_minmax_yearly_trans(transactions)
    get_minmax_monthly_trans(transactions)
    
    
    agg_features=transactions.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

    max_group = get_row_with_max_col_val(transactions, 'amount_rur', 'client_id')[ ["client_id", "small_group"]]
    print(max_group.shape)

    counter_df_train = transactions.groupby(['client_id','small_group'])['amount_rur'].count()
    
    sum_df_train = transactions.groupby(['client_id','small_group'])['amount_rur'].agg(["sum","max"])
    
    
    
    minmax_dayly_transactions_train = \
    transactions.groupby(['client_id','trans_date'])["amount_rur"].agg(['count', 'sum'])\
    .groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

    minmax_dayly_transactions_train.columns = ['client_id', 'daytrans_max', 'daytrans_min', 'daycount_max', 'daycount_min']
    
    
    
    cat_counts_train=counter_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
    cat_counts_train=cat_counts_train.fillna(0)
    cat_counts_train.columns=['sg_count_'+str(i) for i in cat_counts_train.columns]
    
    
    
    cat_sum_train = sum_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values=["sum","max"])
    cat_sum_train = cat_sum_train.fillna(0)
    cat_sum_train.columns=['sg_sum_'+str(i) for i in cat_sum_train.columns]
    
    
    if is_train:
        train=pd.merge(y_train,agg_features,on='client_id')
    
    else:
        train = pd.merge(test_id, cat_counts_train.reset_index(),on='client_id')
        print(train.shape)
        
    train = pd.merge(train, cat_counts_train.reset_index(),on='client_id')
    print(train.shape)


    
    train=pd.merge(train, cat_sum_train.reset_index(),on='client_id')
    print(train.shape)


    train = pd.merge(train, max_group.reset_index()[["client_id", "small_group"]],on='client_id')
    print(train.shape)
    
    
    train=pd.merge(train, minmax_dayly_transactions_train,on='client_id')
    
    
    train = pd.merge(train,get_minmax_monthly_trans(transactions),on='client_id')
    print(train.shape)


    train = pd.merge(train,get_minmax_yearly_trans(transactions),on='client_id')
    print(train.shape)


    
    train = pd.merge(train,get_first_last_transaction_date(transactions).reset_index(),on='client_id')
    print(train.shape)


    
    
    CAT = pd.concat([get_stat(transactions, func='count'), get_stat(transactions, func='sum'),
                        get_stat(transactions, func='std'), get_stat(transactions, func='median')],axis=1)
    
    train = pd.merge(train, CAT.reset_index(), on='client_id')
    
    return train.drop_duplicates(subset="client_id")
    

In [17]:
y_train = REN(pd.read_csv('data/train_target.csv'))

In [18]:
c = preproc_data(transactions_train, True)

(25352, 2)
(25000, 210)
(25000, 602)
(25352, 603)
(25352, 612)
(25352, 617)
(25352, 619)


In [19]:
c.shape, y_train.shape

((25000, 1403), (25000, 9))

In [20]:
transactions_test = reduce_mem_usage(pd.read_csv('data/transactions_test.csv') )
test_id = pd.read_csv('data/test.csv')

Memory usage of dataframe is 668.38 MB
Memory usage after optimization is: 250.64 MB
Decreased by 62.5%


In [21]:
test_id.shape

(25000, 1)

In [22]:
transactions_test  = REN(transactions_test).rename(columns={"amount": "amount_rur"})

In [23]:
transactions_test

,client_id,trans_date,amount_rur,small_group
0,31453,0,20.533001,0
1,3615,0,7.700000,0
2,19950,0,13.441000,1
3,7311,0,8.675000,7
4,36112,0,23.459000,2
...,...,...,...,...
21901475,37497,119,3.850000,7
21901476,35648,119,24.150999,15
21901477,632,119,67.782997,11
21901478,4848,119,13.860000,45


In [25]:
test_id = REN(test_id)

In [26]:
test = preproc_data(transactions_test, False)
test.shape

(25388, 2)
(25000, 197)
(25000, 393)
(25000, 785)
(25388, 786)
(25388, 795)
(25388, 800)
(25388, 802)


(25000, 1586)

In [27]:
common_features = list(set(c.columns).intersection(set(test.columns)))

In [28]:
len(common_features)

1194

In [45]:
TO_PREDICT = ['27', '32', '41', '45', '67', '73', '81', '88']

train_target = c[['client_id'] + TO_PREDICT]
train_data = c[common_features]
X_test = test[common_features]

In [34]:
test.shape

(25000, 1586)

In [35]:
X_test.shape

(25000, 1194)

In [36]:
train_data.shape

(25000, 1194)

In [47]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
import catboost as cb

In [39]:
def local_test(model):
    X_train, X_validation, y_train, y_validation = train_test_split(c.drop('bins',axis=1), c['bins'], test_size=0.3)
    model.fit(
    train_data, y_train,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
    )

# 5000 k iters
seed 228 - 0.6419666666666667 0,6393
seed 123213 - 0.6421666666666667

In [ ]:
def average_score(model):
    score

In [53]:
cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    train_data,
    y_train["27"],
    cv = 4,
    scoring = "accuracy",
    verbose = 1,
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  8.4min finished


array([0.85410334, 0.85330347, 0.85837734, 0.85517683])

In [108]:
param={'objective':'multi:softprob','num_class':4,'n_jobs':16,'seed':42}
model_xgb = xgb.XGBClassifier(**param,n_estimators=600)

In [109]:
cross_val_score(
    model_xgb,
    train,
    y_train,
    cv = 4,
    scoring = "accuracy",
    verbose = 1,
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.6min finished


array([0.62350598, 0.625     , 0.62189451, 0.63114536])

In [110]:
np.mean([0.62350598, 0.625     , 0.62189451, 0.63114536])

0.6253864625

In [86]:
np.mean([0.62124834, 0.62207758, 0.62269164, 0.6290194 ])

0.62375924

In [57]:
np.mean([0.61341301, 0.6186238 , 0.61525176, 0.61506777])

0.615589085

In [32]:
np.mean([0.61580345, 0.61517003, 0.6124618 , 0.61639649])

0.6149579425

In [31]:
np.mean([0.61383631, 0.61813333, 0.61154821, 0.61594879])

0.61486666

# Знаю как реализовать
* - Максимальная транзакция
* - КАтегория максимальной транзакции
* Количество транзакций ниже трешхолда
* Трешхолд для каждого пользователя свой (типо выше среднего)
* Уровень трат выше среднего уровня трат для каждой категории
* Максимальные затраты за 1 день
* Сколько в среднем тратит за 1 день
* Производная от графика (затраты / дни)

# Надо думать

* как считается trans_date ? (Относительно получения карты или абсолютно)
* если абсолютно, то можно попытаться определить дни недели